In [2]:
import os
import sys
sys.path.insert(0, '../src/')
from data_cleaning import raw_json_to_clean_df
from data_transformation import transform_clean_df, get_X_y, get_dist
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron


## 1. Data Collection

We begin by scrapping Wiktionary.org for *feminine*, *masculine*, and *neuter* nouns in *Polish*, *German*, *Spanish*, and *French*.


In [ ]:
# run webscrapper

## 2. Data Cleaning

The raw data in json format must be cleaned: removing nouns with *spaces*, *hyphens*, *numbers*, *abbreviations*, *initials* and finally those that are *proper nouns*.

In [3]:
# read json file and load it as a DataFrame
path = '../data/raw_scraped_data.json'
df = raw_json_to_clean_df(path)

TypeError: '>' not supported between instances of 'method' and 'int'

In [32]:
df.head(5)

,noun,lemma,gender,lang
0,aalen,aalir,masculine,Polish
1,abacysta,abacysta,masculine,Polish
2,abatacept,abatacept,masculine,Polish
3,abatacept,abatacept,masculine,French
4,abatacept,abatacept,masculine,Polish


## 3 Transform and Encode Data
reduce data down to an even amount of examples per language and per gender
encode last 3 letters of each noun

In [34]:
trans_df = transform_clean_df(df)
trans_df.head(5)

/home/capeta/Uni/S2/Ad Morpho/Project/-Morphology-Gender-assignment/notebooks/../src/data/data_transformation.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  reduced_df = df.groupby(['lang', 'gender'])['noun', 'lemma', 'gender', 'lang'].sample(n=3000)
/home/capeta/.local/share/virtualenvs/-Morphology-Gender-assignment-lrLn6GZY/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,noun,lemma,gender,lang,0,1,2,3,4,5,...,2445,2446,2447,2448,2449,2450,2451,2452,2453,2454
0,cafétérie,cafétéria,feminine,French,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,iconographie,iconographie,feminine,French,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,oniomanie,oniomanie,feminine,French,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,baïonnette,baïonnette,feminine,French,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Walcourt,Walcourt,feminine,French,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 4 Train Data

## 4.1 Define X and y

In [35]:
X, y 

## 4.2 Split data into training and testing

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### 4.3 K-NN

In [29]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.8421666666666666

### 4.4 Perceptron

In [30]:
p = Perceptron(random_state=42)
p.fit(X_train, y_train)
p.score(X_test, y_test)

0.8773333333333333